In [ ]:
import io
import os
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
import ipywidgets as widgets
from os import listdir
from os.path import isfile, join

from config import config

%run helpers/parsing.ipynb

# Define global variables for later use
dataset_config = None
filename: str = None
buffer: bytes = None
xl: pd.ExcelFile = None
sheet: pd.DataFrame = None
df: pd.DataFrame = None
column_name: str = None
sheet_name: str = None
gaps_indices = []

sheet_select: widgets.Dropdown = None
column_select: widgets.Dropdown = None


dataset_config = config["datasets_configs"][list(config["datasets_configs"].keys())[0]]

# Create upload status text
upload_status = widgets.HTML(value='')


# Create the file explorer tab
file_select = widgets.Dropdown(
    options=sorted([f for f in listdir(config["upload_dir"]) if isfile(join(config["upload_dir"], f))]),
    value=None
)

def on_file_select_change(evt):
    global file_select, filename, buffer
    file_select.disabled = True
    filename = join(config["upload_dir"], evt.new)
    upload_status.value = 'Upload starting...'
    with open(filename, 'rb') as file:
        buffer = file.read()
        upload_status.value = 'Upload successful'
    parsing_status.value = 'Reading file data...'
    parse_uploaded_file_interactive(filename, buffer)
    file_select.disabled = False
file_select.observe(on_file_select_change, 'value')

file_explorer_form = widgets.VBox(
    children=[
        widgets.HTML(value='Uploaded file selector:'),
        widgets.HBox(children=[
            file_select
        ])
    ]
)


# Create tabs container
display(file_explorer_form)


# Create parsing status text
parsing_status = widgets.HTML(value='')
display(parsing_status)
